# Inserting all users, messages and channels into a MongoDB database

In [52]:
import os
import sys
import pprint
import uuid
import json
import pandas as pd
import os, sys

# Add parent directory to path to import modules from src
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)


from pymongo import MongoClient
from src.loader import SlackDataLoader


slack_data_path = os.path.abspath('../anonymized')

In [9]:
class DBWithSchema:
    def __init__(self) -> None:
        self.client = MongoClient("mongodb://localhost:27017/")
        self.db = self.client["SlackDb"]

        self.user_validator = {
            "$jsonSchema": {
                "bsonType": "object",
                "required": ["user_id", "real_name"],
                "properties": {
                    "user_id": {
                        "bsonType": "string",
                        "description": "must be a string and is required"
                    },
                    "real_name": {
                        "bsonType": "string",
                        "description": "must be a string and is required"
                    }
                }
            }
        }

        self.channel_validator = {
            "$jsonSchema": {
                "bsonType": "object",
                "required": ["channel_id", "name"],
                "properties": {
                    "channel_id": {
                        "bsonType": "string",
                        "description": "must be a string and is required"
                    },
                    "name": {
                        "bsonType": "string",
                        "description": "must be a string and is required"
                    }
                }
            }
        }

        self.message_validator = {
            "$jsonSchema": {
                "bsonType": "object",
                "required": ["ts", "msg_id", "text", "user_id", "channel_id", "mentions", "reactions", "replies"],
                "properties": {
                    "ts": {
                        "bsonType": "string",
                        "description": "must be a string and is required"
                    },

                    "msg_id": {
                        "bsonType": "string",
                        "description": "must be a string and is required"
                    },
                    "text": {
                        "bsonType": "string",
                        "description": "must be a string and is required"
                    },
                    "user_id": {
                        "bsonType": "string",
                        "description": "must be a string and is required"
                    },
                    "channel_id": {
                        "bsonType": "string",
                        "description": "must be a string and is required"
                    },
                    "mentions": {
                        'bsonType': ['array', 'null'],

                        "items": {
                            "bsonType": "string"
                        },
                        "description": "must be an array of strings"
                    },
                    "reactions": {
                        'bsonType': ['array', 'null'],

                        "items": {
                            "bsonType": "object",
                            "required": ["name", "users", "count"],
                            "properties": {
                                "name": {
                                    "bsonType": "string",
                                    "description": "must be a string and is required"
                                },
                                "users": {
                                    "bsonType": "array",
                                    "items": {
                                        "bsonType": "string"
                                    },
                                    "description": "must be an array of strings"
                                },
                                "count": {
                                    "bsonType": "number",
                                    "description": "must be a number and is required"
                                }
                            }
                        }
                    },
                    "replies": {
                        'bsonType': ['array', 'null'],

                        "items": {
                            "bsonType": "object",
                            "required": ["user", "ts"],
                            "properties": {
                                "user": {
                                    "bsonType": "string",
                                    "description": "must be a string and is required"
                                },
                                "ts": {
                                    "bsonType": "string",
                                    "description": "must be a string and is required"
                                }
                            }
                        }
                    }
                }
            }
        }


        try:
            self.db.create_collection("users")
            self.db.create_collection("channels")
            self.db.create_collection("messages")
        except Exception as e:
            print("error: ", e)

        self.db.command("collMod", "users", validator=self.user_validator)
        self.db.command("collMod", "channels", validator=self.channel_validator)
        self.db.command("collMod", "messages", validator=self.message_validator)



    def list_collections(self):
        return self.db.list_collection_names()
    
    def get_validation(self, collection_name: str) -> dict:
        self.check_if_collection_exist(collection_name)
        return self.db.get_collection(collection_name).options()
    
    def check_if_collection_exist(self, collection_name: str):
        if not self.list_collections().__contains__(collection_name):
            raise Exception(f"Collection, {collection_name} not found.")

    def insert_to_collection(self, collection_name, data):
        self.check_if_collection_exist(collection_name)
        collection = self.db[collection_name]
        return collection.insert_one(data)

    def insert_many_to_collection(self, collection_name, data):
        self.check_if_collection_exist(collection_name)
        result = self.db[collection_name].insert_many(data)
        return result.inserted_ids



    def find_all(self, collection_name):
        self.check_if_collection_exist(collection_name)
        return self.db[collection_name].find()

    def find(self, collection_name, key, value):
        self.check_if_collection_exist(collection_name)
        return self.db[collection_name].find({key: value})
    
    def find_by_id(self, collection_name, _id):
        self.check_if_collection_exist(collection_name)
        return self.db[collection_name].find

    def find_one(self, collection_name, key, value):
        self.check_if_collection_exist(collection_name)
        return self.db[collection_name].find_one({key: value})

In [11]:
database = DBWithSchema()

error:  collection users already exists


# Adding all users from the slack workspace to the database

In [16]:
users = SlackDataLoader(slack_data_path).users

formatted_users = [{"user_id": user["id"], "real_name": user["real_name"]} for user in users]

database.insert_many_to_collection('users', formatted_users)

[ObjectId('6569a9b914c3b0d37a8165ec'),
 ObjectId('6569a9b914c3b0d37a8165ed'),
 ObjectId('6569a9b914c3b0d37a8165ee'),
 ObjectId('6569a9b914c3b0d37a8165ef'),
 ObjectId('6569a9b914c3b0d37a8165f0'),
 ObjectId('6569a9b914c3b0d37a8165f1'),
 ObjectId('6569a9b914c3b0d37a8165f2'),
 ObjectId('6569a9b914c3b0d37a8165f3'),
 ObjectId('6569a9b914c3b0d37a8165f4'),
 ObjectId('6569a9b914c3b0d37a8165f5'),
 ObjectId('6569a9b914c3b0d37a8165f6'),
 ObjectId('6569a9b914c3b0d37a8165f7'),
 ObjectId('6569a9b914c3b0d37a8165f8'),
 ObjectId('6569a9b914c3b0d37a8165f9'),
 ObjectId('6569a9b914c3b0d37a8165fa'),
 ObjectId('6569a9b914c3b0d37a8165fb'),
 ObjectId('6569a9b914c3b0d37a8165fc'),
 ObjectId('6569a9b914c3b0d37a8165fd'),
 ObjectId('6569a9b914c3b0d37a8165fe'),
 ObjectId('6569a9b914c3b0d37a8165ff'),
 ObjectId('6569a9b914c3b0d37a816600'),
 ObjectId('6569a9b914c3b0d37a816601'),
 ObjectId('6569a9b914c3b0d37a816602'),
 ObjectId('6569a9b914c3b0d37a816603'),
 ObjectId('6569a9b914c3b0d37a816604'),
 ObjectId('6569a9b914c3b0

In [25]:
#first five user recorded in the database 
records = database.find_all('users')
for record in records[:5]:
    pprint.pprint(record)

{'_id': ObjectId('6569a9b914c3b0d37a8165ec'),
 'real_name': 'Rachel Mann',
 'user_id': 'U03T89ACUUW'}
{'_id': ObjectId('6569a9b914c3b0d37a8165ed'),
 'real_name': 'Bridget Glover',
 'user_id': 'U03TEPYRM2P'}
{'_id': ObjectId('6569a9b914c3b0d37a8165ee'),
 'real_name': 'Victor Mayo',
 'user_id': 'U03TNP8Q8CT'}
{'_id': ObjectId('6569a9b914c3b0d37a8165ef'),
 'real_name': 'Brian Flores',
 'user_id': 'U03TT5KEYCF'}
{'_id': ObjectId('6569a9b914c3b0d37a8165f0'),
 'real_name': 'Joanne Thomas',
 'user_id': 'U03TX2VN6H5'}


# Adding all channels of the slack workspace to the database

In [28]:
all_channels = SlackDataLoader(slack_data_path).channels

formatted_channels = [{"channel_id": channel["id"], "name": channel["name"]} for channel in all_channels]

database.insert_many_to_collection('channels', formatted_channels)


[ObjectId('6569ab7414c3b0d37a816633'),
 ObjectId('6569ab7414c3b0d37a816634'),
 ObjectId('6569ab7414c3b0d37a816635'),
 ObjectId('6569ab7414c3b0d37a816636'),
 ObjectId('6569ab7414c3b0d37a816637'),
 ObjectId('6569ab7414c3b0d37a816638'),
 ObjectId('6569ab7414c3b0d37a816639'),
 ObjectId('6569ab7414c3b0d37a81663a'),
 ObjectId('6569ab7414c3b0d37a81663b'),
 ObjectId('6569ab7414c3b0d37a81663c'),
 ObjectId('6569ab7414c3b0d37a81663d'),
 ObjectId('6569ab7414c3b0d37a81663e'),
 ObjectId('6569ab7414c3b0d37a81663f'),
 ObjectId('6569ab7414c3b0d37a816640'),
 ObjectId('6569ab7414c3b0d37a816641'),
 ObjectId('6569ab7414c3b0d37a816642'),
 ObjectId('6569ab7414c3b0d37a816643'),
 ObjectId('6569ab7414c3b0d37a816644'),
 ObjectId('6569ab7414c3b0d37a816645'),
 ObjectId('6569ab7414c3b0d37a816646'),
 ObjectId('6569ab7414c3b0d37a816647'),
 ObjectId('6569ab7414c3b0d37a816648'),
 ObjectId('6569ab7414c3b0d37a816649'),
 ObjectId('6569ab7414c3b0d37a81664a'),
 ObjectId('6569ab7414c3b0d37a81664b'),
 ObjectId('6569ab7414c3b0

In [29]:
#first five channels recorded in the database 
records = database.find_all('channels')
for record in records[:5]:
    pprint.pprint(record)

{'_id': ObjectId('6569ab7414c3b0d37a816633'),
 'channel_id': 'C03T0APHX63',
 'name': 'all-community-building'}
{'_id': ObjectId('6569ab7414c3b0d37a816634'),
 'channel_id': 'C03T0AX4K6K',
 'name': 'all-technical-support'}
{'_id': ObjectId('6569ab7414c3b0d37a816635'),
 'channel_id': 'C03T89KDGA2',
 'name': 'all-career-exercises'}
{'_id': ObjectId('6569ab7414c3b0d37a816636'),
 'channel_id': 'C03T89PMJKG',
 'name': 'all-resources'}
{'_id': ObjectId('6569ab7414c3b0d37a816637'),
 'channel_id': 'C03TBUCU4UD',
 'name': 'random'}


# Adding all messages in the slack workspace to the database

In [63]:
def remove_duplicates(json_files, key):
    seen = set()
    unique_json_files = []

    for json_file in json_files:
        json_key = json_file.get(key)
        if json_key not in seen:
            unique_json_files.append(json_file)
            seen.add(json_key)

    return unique_json_files

def get_messages_dict(msgs):
    msg_list = {
            "msg_id":[],
            "text":[],
            "user":[],
            "mentions":[],
            "reactions":[],
            "replies":[],
            "replies_to":[],
            "ts":[],
            }


    for msg in msgs:
        if "subtype" not in msg:
            try:
                msg_list["msg_id"].append(msg["client_msg_id"])
            except:
                msg_list["msg_id"].append(None)
            msg_list["text"].append(msg["text"])

            msg_list["user"].append(msg["user"])
            msg_list["ts"].append(msg["ts"])

            
            if "reactions" in msg:
                msg_list["reactions"].append(msg["reactions"])
            else:

                msg_list["reactions"].append(None)

            if "parent_user_id" in msg:
                msg_list["replies_to"].append(msg["ts"])
            else:
                msg_list["replies_to"].append(None)

            if "thread_ts" in msg and "reply_users" in msg:
                msg_list["replies"].append(msg["replies"])
            else:
                msg_list["replies"].append(None)
            
            if "blocks" in msg and msg["blocks"]:
                mention_list = []
                
                for blk in msg["blocks"]:
                    if "elements" in blk:
                        for elm in blk["elements"]:
                            if "elements" in elm:
                                for elm_ in elm["elements"]:
                                    
                                    if "type" in elm_:
                                      
                                        if elm_["type"] == "user":
                                            mention_list.append(elm_["user_id"])
                                       


                msg_list["mentions"].append(mention_list)
            else:
                msg_list["mentions"].append(None)
    
    return msg_list



def get_messages_from_channel(channel_path):
    json_files = [
        f"{channel_path}/{pos_json}" 
        for pos_json in os.listdir(channel_path) 
        if pos_json.endswith('.json')
    ]    
    combined = []

    for json_file in json_files:
        with open(json_file, 'r', encoding="utf8") as slack_data:
            json_content = json.load(slack_data)
            combined.extend(json_content)
        
    # print(combined[0])
    msg_list = get_messages_dict(combined)
    df = pd.DataFrame(msg_list)
    
    return df

In [65]:
all_messages = []

# print(SlackDataLoader(slack_data_path).channels[0])
for channel in SlackDataLoader(slack_data_path).channels:

    messages = get_messages_from_channel(f"../anonymized/{channel['name']}")

    formatted_messages = messages.apply(lambda msg: {
        "msg_id": msg["msg_id"] if msg["msg_id"] else str(uuid.uuid4()),
        "text": msg["text"],
        "user_id": msg["user"],
        "mentions": msg["mentions"] if msg["mentions"] else [],
        "reactions": msg["reactions"] if msg["reactions"] else [],
        "replies": msg["replies"] if msg["replies"] else [],
        "ts": msg["ts"],
        "channel_id": channel["id"]
    }, axis=1)

    all_messages.extend(formatted_messages.to_list())


all_messages = remove_duplicates(all_messages, 'msg_id')

database.insert_many_to_collection('messages', all_messages)


{'client_msg_id': 'aaea2ec8-1461-4705-98c0-fda055acf96e', 'type': 'message', 'text': 'morning boss', 'user': 'U03UVHCV6KB', 'ts': '1666940798.397899', 'blocks': [{'type': 'rich_text', 'block_id': '8Nn0E', 'elements': [{'type': 'rich_text_section', 'elements': [{'type': 'text', 'text': 'morning boss'}]}]}], 'team': 'T03U4J8HMUG', 'user_team': 'T03U4J8HMUG', 'source_team': 'T03U4J8HMUG', 'user_profile': {'avatar_hash': 'gc9619121f99', 'image_72': 'https://secure.gravatar.com/avatar/c9619121f998ee4a7e46177f2a5729de.jpg?s=72&d=https%3A%2F%2Fa.slack-edge.com%2Fdf10d%2Fimg%2Favatars%2Fava_0002-72.png', 'first_name': 'Gregory', 'real_name': 'Gregory Bowman', 'display_name': 'Gregory Bowman', 'team': 'T03U4J8HMUG', 'name': 'Gregory', 'is_restricted': False, 'is_ultra_restricted': False}, 'attachments': None}
{'client_msg_id': 'bd822d64-f723-4be4-a79e-26d5e69f0d94', 'type': 'message', 'text': "good morning,\ni'm not able to access the tenx platform with my chrome navigator. i'm getting error si

[ObjectId('6569b5a114c3b0d37a81b057'),
 ObjectId('6569b5a114c3b0d37a81b058'),
 ObjectId('6569b5a114c3b0d37a81b059'),
 ObjectId('6569b5a114c3b0d37a81b05a'),
 ObjectId('6569b5a114c3b0d37a81b05b'),
 ObjectId('6569b5a114c3b0d37a81b05c'),
 ObjectId('6569b5a114c3b0d37a81b05d'),
 ObjectId('6569b5a114c3b0d37a81b05e'),
 ObjectId('6569b5a114c3b0d37a81b05f'),
 ObjectId('6569b5a114c3b0d37a81b060'),
 ObjectId('6569b5a114c3b0d37a81b061'),
 ObjectId('6569b5a114c3b0d37a81b062'),
 ObjectId('6569b5a114c3b0d37a81b063'),
 ObjectId('6569b5a114c3b0d37a81b064'),
 ObjectId('6569b5a114c3b0d37a81b065'),
 ObjectId('6569b5a114c3b0d37a81b066'),
 ObjectId('6569b5a114c3b0d37a81b067'),
 ObjectId('6569b5a114c3b0d37a81b068'),
 ObjectId('6569b5a114c3b0d37a81b069'),
 ObjectId('6569b5a114c3b0d37a81b06a'),
 ObjectId('6569b5a114c3b0d37a81b06b'),
 ObjectId('6569b5a114c3b0d37a81b06c'),
 ObjectId('6569b5a114c3b0d37a81b06d'),
 ObjectId('6569b5a114c3b0d37a81b06e'),
 ObjectId('6569b5a114c3b0d37a81b06f'),
 ObjectId('6569b5a114c3b0

In [66]:
#first five messages recorded in the database 
records = database.find_all('messages')
for record in records[:5]:
    pprint.pprint(record)

{'_id': ObjectId('6569b59314c3b0d37a81665a'),
 'channel_id': 'C03T0APHX63',
 'mentions': [],
 'msg_id': 'aaea2ec8-1461-4705-98c0-fda055acf96e',
 'reactions': [],
 'replies': [],
 'text': 'morning boss',
 'ts': '1666940798.397899',
 'user_id': 'U03UVHCV6KB'}
{'_id': ObjectId('6569b59314c3b0d37a81665b'),
 'channel_id': 'C03T0APHX63',
 'mentions': ['U03UUN8M4RX'],
 'msg_id': '2c87b913-e78f-4f9b-bc3e-c1a524a0fe0a',
 'reactions': [{'count': 8,
                'name': 'raised_hands',
                'users': ['U03U1FNPEUX',
                          'U03UD68RQH3',
                          'U03UG32J3PC',
                          'U03UG0SFHGT',
                          'U03UG1Z21JP',
                          'U03UFV7HFNF',
                          'U03U1HAG9TR',
                          'U03UJH1EQQL']},
               {'count': 1,
                'name': 'raised_hands::skin-tone-4',
                'users': ['U03UG4Q7V42']},
               {'count': 1,
                'name': 'raised_han